![](guro.png)

# Ayudantia 3: Gurobi

### Guía de instalación:

1. Tener instalado python
2. Registrarse en gurobi con el mail UC [https://www.gurobi.com/es/]
3. Descargar Gurobi [http://www.gurobi.com/downloads/gurobi-optimizer]
4. Pedir Licencia académica [http://www.gurobi.com/downloads/licenses/license-center]
5. Activar la licencia usando la red wifi UC (debe hacerse desde la red de la UC). Siga la documentación [http://www.gurobi.com/documentation/]
6. Instalar Gurobi en Anaconda*.
7. Seguir instrucciones de instalación en [http://www.gurobi.com/documentation/]


Crear un modelo de gurobi requiere de 6 pasos

* __Crear modelo:__ m = Model( )
* __Agregar Variables:__ x = m.addVar(...)
* __Actualizar cambios:__ m.update( )
* __Fijar func. objetivo:__ m.setObjective(...)
* __Agregar restricciones:__ m.addConstr(...)
* __Optimizar!:__ m.optimize( )

In [95]:
from gurobipy import *
from random import randint, choice

# Gurobi infeasible model

* Resolver un modelo de asignación de trabajadores a turnos
* Este modelo en particular es infactible, intentaremos determinar a que se debe.

In [96]:
m = Model("planificacion")

semanas = 2
dias = ["L","M","W","J","V","S","D"]

turnos = [v+str(i+1) for i, v in enumerate(dias*semanas)] # ["L1","M2","W3","J4","V5","S6","D7","L8"...]
trabajadores = ["Tomas", "Javiera", "Marcelo", "Jorge", "Ana", "Camila", "Diego"]


In [97]:
# requerimientos por turno
requerimientos = {}
for i in turnos:
    requerimientos[i] = randint(2,7)
print(requerimientos)

{'L1': 6, 'M2': 6, 'W3': 4, 'J4': 3, 'V5': 2, 'S6': 4, 'D7': 6, 'L8': 3, 'M9': 2, 'W10': 6, 'J11': 7, 'V12': 4, 'S13': 2, 'D14': 2}


In [98]:
# pagos asociados
pago = {"Tomas":  10, 
        "Javiera":12, 
        "Marcelo":10, 
        "Jorge":  8, 
        "Ana":    8, 
        "Camila": 9, 
        "Diego":  11}

## Disponibilidad de los trabajadores

In [99]:
# optimizamos solo los dias disponibles

disponibilidad = []
for trabajador in trabajadores:
    count = 1
    dias_disponibles = randint(5,13)
    while(count <= dias_disponibles):
        while True:
            eleccion = choice(turnos)
            if (trabajador,eleccion) not in disponibilidad:
                disponibilidad.append((trabajador,eleccion))
                break
        count += 1
disponibilidad = tuplelist(disponibilidad)
print(disponibilidad)

<gurobi.tuplelist (58 tuples, 2 values each):
 ( Tomas   , J4  )
 ( Tomas   , J11 )
 ( Tomas   , L1  )
 ( Tomas   , M2  )
 ( Tomas   , V12 )
 ( Tomas   , D7  )
 ( Tomas   , S6  )
 ( Tomas   , S13 )
 ( Tomas   , M9  )
 ( Tomas   , L8  )
 ( Javiera , L8  )
 ( Javiera , L1  )
 ( Javiera , V12 )
 ( Javiera , M2  )
 ( Javiera , V5  )
 ( Marcelo , D14 )
 ( Marcelo , W3  )
 ( Marcelo , V12 )
 ( Marcelo , D7  )
 ( Marcelo , S13 )
 ( Marcelo , J11 )
 ( Marcelo , L1  )
 ( Marcelo , S6  )
 ( Jorge   , S13 )
 ( Jorge   , D14 )
 ( Jorge   , S6  )
 ( Jorge   , M2  )
 ( Jorge   , W10 )
 ( Ana     , S13 )
 ( Ana     , S6  )
 ( Ana     , M2  )
 ( Ana     , V5  )
 ( Ana     , M9  )
 ( Ana     , W3  )
 ( Ana     , W10 )
 ( Ana     , V12 )
 ( Ana     , L8  )
 ( Camila  , S6  )
 ( Camila  , J11 )
 ( Camila  , W10 )
 ( Camila  , W3  )
 ( Camila  , D7  )
 ( Camila  , L1  )
 ( Camila  , M2  )
 ( Camila  , L8  )
 ( Camila  , M9  )
 ( Camila  , V12 )
 ( Camila  , S13 )
 ( Diego   , J11 )
 ( Diego   , S13 )
 ( D

## Variables de decisión

Variable binaria x\[t,s\] = 1 si el trabajador t es asignado al turno s

In [100]:
x = m.addVars(disponibilidad, ub=1, name="x")

## Función Objetivo

Minimizar el salario total

In [101]:
m.setObjective(sum(pago[t]*x[t,s] for t,s in disponibilidad), GRB.MINIMIZE)

## Restricciones asociadas al problema

Por cada turno, el numero de trabajadores debe ser igual al que se necesita para ese turno.

In [102]:
# La funcion sum es aplicable a tuplas, esta retorna la suma de los valores asociados a determinadas keys.
# El '*' indica que cualquier valor es aceptado en ese campo, si no colocamos argumentos entonces se retorna la suma
# de todos los valores en la tupla
# https://www.gurobi.com/documentation/8.1/refman/py_tupledict_sum.html
reqCts = m.addConstrs((x.sum('*', s) == requerimientos[s] for s in turnos),"_")

In [103]:
# right-hand side

m.optimize()

Optimize a model with 14 rows, 58 columns and 58 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 7e+00]
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds
Infeasible model


In [104]:
m.write("file_infeasible.lp")

# Modificando un modelo

* Cambiar coeficientes (objective, RHS, Matrix, Bounds)
* Cambiar Variables (continuas o enteras)
* Agregar/Borrar Varaibles o restricciones
* feasRelaxS permite crear una relajación factible del modelo


In [105]:
# Argumentos 
# modelo que minimiza la cantidad donde la solucion viola los limites y las restricciones del modelo original
# relaxobjtype: 0 para que minimizar la suma de las magnitudes de limites y restricciones violados
# relaxobjtype: 1 para que minimizar la suma de los cuadrados de limites y restricciones violados
# relaxobjtype: 2 para que minimizar la suma total de limites y restricciones violados

# si una restriccion fuera violada por 2, la funcion contribuye en 2 a la relajacion con 0, 2*2 en 1 y 1 en 2.

# minrelax: booleano que controla el tipo de relajacion creado false minimiza e costo de la violacion, true encuentra una solucion que minimiza el obejtivo original
# solo a traves de estas soluciones minimiza el costo de la violacion (costoso)

# vrelax: indica cuales variable pueden ser relajadas
# crelax: indica que restricciones pueden ser relajadas
orignumvars = m.NumVars
m.feasRelaxS(0, False,False, True)
m.optimize()

m.printAttr('X',"Art*")


Optimize a model with 14 rows, 86 columns and 86 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 7e+00]
Presolve removed 8 rows and 68 columns
Presolve time: 0.04s
Presolved: 6 rows, 18 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2000000e+01   0.000000e+00   0.000000e+00      0s
       0    1.2000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.06 seconds
Optimal objective  1.200000000e+01

    Variable            X 
-------------------------
     ArtP_L1            1 
     ArtP_M2            1 
     ArtP_J4            2 
     ArtP_D7            2 
    ArtP_W10            3 
    ArtP_J11            3 


In [107]:
print(m.objVal)

12.0


In [109]:
m.printAttr('X')


    Variable            X 
-------------------------
 x[Tomas,J4]            1 
x[Tomas,J11]            1 
 x[Tomas,L1]            1 
 x[Tomas,M2]            1 
 x[Tomas,D7]            1 
 x[Tomas,S6]            1 
x[Tomas,S13]            1 
 x[Tomas,M9]            1 
 x[Tomas,L8]            1 
x[Javiera,L8]            1 
x[Javiera,L1]            1 
x[Javiera,V12]            1 
x[Javiera,M2]            1 
x[Javiera,V5]            1 
x[Marcelo,D14]            1 
x[Marcelo,W3]            1 
x[Marcelo,V12]            1 
x[Marcelo,D7]            1 
x[Marcelo,J11]            1 
x[Marcelo,L1]            1 
x[Jorge,D14]            1 
 x[Jorge,S6]            1 
 x[Jorge,M2]            1 
x[Jorge,W10]            1 
  x[Ana,S13]            1 
   x[Ana,S6]            1 
   x[Ana,M2]            1 
   x[Ana,V5]            1 
   x[Ana,W3]            1 
  x[Ana,W10]            1 
   x[Ana,L8]            1 
x[Camila,J11]            1 
x[Camila,W10]            1 
x[Camila,W3]            1 
x[Camila,D7

In [84]:
m.write("file.lp")

In [108]:
for v in m.getVars():
    print(v.x)

1.0
1.0
1.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
1.0
0.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
0.0
1.0
0.0
1.0
0.0
1.0
0.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0
0.0
1.0
0.0
0.0
0.0
2.0
0.0
0.0
0.0
0.0
0.0
2.0
0.0
0.0
0.0
0.0
0.0
3.0
0.0
3.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [106]:
orignumvars

58